# Input parameters

# Code

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import NoSuchElementException
import sys
import ipywidgets as widgets
from IPython.display import display, HTML

def alertBox(msg):
        js = "<script>alert('%s');</script>"%(msg,)
        display(HTML(js))

class FirefoxTrip:
    def __init__(self):
        self.data=pd.DataFrame()
        #driver
        self.driver = webdriver.Firefox()
        self.driver.set_window_size(1024,768)
        self.driver.get("https://www.tripadvisor.com")
        alertBox("로그인을 먼저 하세요.")
    def visit(self,url):
        self.driver.get(url)
    def get_links(self):
        status=True
        try:
            eles=self.driver.find_elements_by_css_selector('a.ui_header_link')
            output=[ele.get_attribute('href') for ele in eles]
        except NoSuchElementException:
            status=False
        if not status:
            return status
        self.current_links=output
        if len(output)<=0:
            status=False
        else:
            temp=pd.DataFrame({"site":"CassBatllo","reviwer_profile":output})
            self.data=pd.concat([self.data,temp])
        return status
    def reset_data(self):
        self.data=pd.DataFrame()
    def export(self,filename):
        self.data.to_excel(filename,index=False)
class ReviewerData:
    def __init__(self,dfile):
        self.dfile=dfile
        self.read()
    def read(self):
        self.df=pd.read_excel(self.dfile)
    def get(self,idx):
        df=self.df
        result=None
        try:
            result=df.loc[df['rowid']==idx,'reviwer_profile'].values[0]
        except IndexError:
            result=None
        return result
    def report(self):
        print("Size of profile = "%(self.df.shape[0],))
class Worker:
    def __init__(self,firefox_obj,reviewer_profile_data):
        self.firefox=firefox_obj
        self.reviewer_profile_data=reviewer_profile_data
    def visit_profile(self,idx):
        url=self.reviewer_profile_data.get(idx)
        if url is not None:
            self.firefox.visit(url)
        else:
            alertBox("프로파일이 존재하지 않습니다.")
    def open_msg_box(self):
        w=self.firefox.driver
        try:
            w.find_element_by_css_selector('[class*=social-message-MessageBlock]').click()
        except NoSuchElementException:
            pass
    def type_subject(self,msg):
        w=self.firefox.driver
        try:
            w.find_element_by_css_selector('input[name*=subject]').send_keys(msg)
        except NoSuchElementException:
            pass
    def type_message(self,msg):
        w=self.firefox.driver
        try:
            w.find_element_by_css_selector('textarea[name*=body]').send_keys(msg)
        except NoSuchElementException:
            pass

def on_button_clicked1(b):
    output.clear_output()
    output.append_stdout('Step 1..')
    worker.visit_profile(profile_index)
def on_button_clicked2(b):
    worker.open_msg_box()
    output.append_stdout('Step 2..')
def on_button_clicked3(b):
    global profile_index
    worker.type_subject(subject_msg)
    worker.type_message(body_msg)
    output.append_stdout("...Processing!...at profile #%d"%(profile_index,))
    output.append_stdout(".. OK.")
    profile_index+=1   

# 실행 부분

### 보내는 메시지 내용은 아래에서 수정합니다.

제목 부분 수정은 아래에서 합니다.

In [4]:
subject_msg="The CASA Battlo AR Guide Survey"

본문에 포함될 메시지는 아래에서 수정합니다.

In [5]:
body_msg='''Dear Survey Respondent,

 Hello, I am a Ph.D. student in Hospitality and Tourism Management at Kyung Hee University in South Korea.
I kindly request you to answer my survey questionnaire, if you respond to the survey questionnaire, you will be paid off USD $ 5 Amazon mobile coupon. 
Please be sure to write up your email address on the last page of the survey. I will take care of your privacy very carefully and will not share your individual information to any third party.
Thank you for your cooperation in advance.
Wish you all the best,

Eunji Lee, PhD student. 
-Researcher name : Eunji Lee
-Email : edreamerj@khu.ac.kr
-Survey URL : https://docs.google.com/forms/d/e/1FAIpQLSehSlJ14kGXpeTfktWwmOnPepj8AF3ER_FfzAO4JUDg6pDtBA/viewform?usp=sf_link
'''